# MOUNT DATA STORAGE FROM AZURE DATALAKE GEN2 TO DATABRICKS WORKSPACE

## Import all secret credentials from local system environment variables using `os` Python built-in library
See <a href="https://stackoverflow.com/questions/56856005/how-to-set-environment-variable-in-databricks#:~:text=You%20can%20set%20environment%20variable,Variables%20%3D%3E%20Confirm%20and%20Restart">How to set environment variable in databricks?</a> on Stack Overflow

In [ ]:
import os
ADB_APPLICATION_ID = os.getenv('ADB_APPLICATION_ID')      # Databricks Application (client) ID
ADB_DIRECTORY_ID = os.getenv('ADB_DIRECTORY_ID')          # Databricks Directory (tenant) ID
ADB_SECRET_ID = os.getenv('ADB_SECRET_ID')                # Databricks Secret ID, created in App Services on Azure portal
STORAGE_ACCOUNT_NAME = os.getenv('STORAGE_ACCOUNT_NAME')  # Azure Data Lake Storage account name 

## Mount `bronze`, `silver`, and `gold` containers
Source : https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts

In [ ]:
CONTAINERS = ['bronze','silver','gold'] # List of containers to be mounted 

In [ ]:
for container in CONTAINERS:
  configs = {"fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{ADB_APPLICATION_ID}",
            "fs.azure.account.oauth2.client.secret": f"{ADB_SECRET_ID}",
            "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{ADB_DIRECTORY_ID}/oauth2/token"}
  
  MOUNT_NAME = container
  MOUNT_POINT = f"/mnt/{MOUNT_NAME}/"

  # Optionally, you can add <directory-name> to the source URI of your mount point.
  dbutils.fs.mount(
    source = f"abfss://{container}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/",
    mount_point = MOUNT_POINT,
    extra_configs = configs)
  
  print(f"Container : {container} was successfully mounted!")

Container : bronze was successfully mounted!
Container : silver was successfully mounted!
Container : gold was successfully mounted!


### Check all the mount points in Databricks

In [ ]:
dbutils.fs.mounts()

### Check if the mounts were succesful or not

In [ ]:
# Access all folders and files inside ADLS Gen2 in the specified container
# dbutils.fs.ls(f"/mnt/{MOUNT_NAME}/")

In [ ]:
%fs
ls "/mnt/bronze/"

path,name,size,modificationTime
dbfs:/mnt/bronze/dataset_URL.json,dataset_URL.json,650,1698768070000


In [ ]:
%fs
ls "/mnt/gold/"

In [ ]:
%fs
ls "/mnt/silver/"

### Unmount a moint point location
If the mount point still in use, uncomment the below.

In [ ]:
dbutils.fs.unmount(f"/mnt/bronze/")

/mnt/bronze/ has been unmounted.


True

In [ ]:
dbutils.fs.unmount(f"/mnt/silver/")

/mnt/silver/ has been unmounted.


True

In [ ]:
dbutils.fs.unmount(f"/mnt/gold/")

/mnt/gold/ has been unmounted.


True